## Imports

In [1]:
import os
import pickle

import langchain.chains
import langchain.prompts
from yandex_chain import YandexLLM, YandexGPTModel
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder

from langchain.schema.runnable import RunnableMap
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.schema import format_document
from operator import itemgetter

## Загрузка данных для ретрива

- Загрузим с локального диска FAISS db, данные для BM25
- Соберем в EnsembleRetriever
- Загрузим модель реранка `BAAI/bge-reranker-v2-m3` (с диска или с интернета)
- Создадим (с реранком и без) финальный ретривер

#### Загрузка `FAISS` (проверить cpu/gpu)

In [105]:
name_db = f"FAISS_CPU_USER-bge-m3_RecursiveCharacter_CHUNK_SIZE_512"
load_path_db = f"../../data/vector_dbs/{name_db}"

with open(load_path_db, 'rb') as f:
    db = pickle.load(f)

#### Загрузка данных для `BM25Retriever`

In [106]:
name_chunked_documents_with_page_content = f"chunked_documents_with_page_content_RecursiveCharacter_CHUNK_SIZE_512"
load_path_chunked_documents_with_page_content = f"../../data/chunked_documents/{name_chunked_documents_with_page_content}"

with open(f'{load_path_chunked_documents_with_page_content}.pkl', 'rb') as f:
    chunked_documents_with_page_content = pickle.load(f)  

In [107]:
bm25_retriever = BM25Retriever.from_documents(
    chunked_documents_with_page_content)

### EnsembleRetriever

In [191]:
num_docs = 3 # как аргумент фунции

retriever = db.as_retriever(search_kwargs={"k": num_docs})
bm25_retriever.k = num_docs

# initialize the ensemble retriever
vector_database = EnsembleRetriever(
    retrievers=[bm25_retriever, retriever], weights=[0.5, 0.5] # You can adjust the weight of each retriever in the EnsembleRetriever
)

### Загрузка модели реранкера и создание финального ретривера

In [192]:
reranker_model_name = "BAAI/bge-reranker-v2-m3"
num_docs_rerank = 1 # как аргумент фунции

model = HuggingFaceCrossEncoder(model_name=reranker_model_name)
compressor = CrossEncoderReranker(model=model, top_n=num_docs_rerank)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=vector_database)

### Langchain с YandexGPT

In [187]:
os.environ['folder_id'] = ""
os.environ['api_key'] = ""

In [196]:
llm = YandexLLM(folder_id=os.environ['folder_id'],
                api_key=os.environ['api_key'],
                model=YandexGPTModel.Pro)

prompt = """
Ты юридический консультант.
Посмотри на текст ниже и ответь на вопрос, используя информацию из этого текста.
Если текст не относится к вопросу, составь ответ не учитывая этот текст.
Если в тексте указана статья, в ответе в самом начале укажи это.
Ответ составь в формате простого текста, не используя markdown
Текст:
-----
{context}
-----
Вопрос:
{question}"""

prompt = langchain.prompts.PromptTemplate(
    template=prompt, input_variables=["context", "question"]
)

def replace_non_breaking_spaces(text):
    return text.replace('\xa0', ' ')

def replace_underscore(text):
    return text.replace('_', ' ')

def join_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Создаём цепочку
chain = (
    {"context": compression_retriever | join_docs | replace_non_breaking_spaces | replace_underscore, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

q = 'Как получить ипотеку'

chain.invoke(q)

'Чтобы получить ипотеку, нужно выполнить ряд действий:\n\n1. Выбрать подходящую ипотечную программу и условия в банке или иной кредитной организации.\n2. Подготовить необходимые документы, включая подтверждение платёжеспособности и занятости.\n3. Обратиться в банк или к иному кредитору с заявкой на ипотечный кредит.\n4. Дождаться решения о предоставлении кредита и, если оно положительное, выбрать недвижимость для покупки.\n5. Провести оценку стоимости недвижимости и оформить договор купли-продажи с продавцом.\n6. Оформить договор страхования недвижимости и жизни (если это требуется условиями банка).\n7. Собрать пакет документов для государственной регистрации ипотеки и подать его в Росреестр (в зависимости от условий могут потребоваться различные документы: заявления залогодателя и залогодержателя, договор управления залогом, договоры, на основании которых возникло обеспечиваемое ипотекой обязательство).  \n\nЭто общие шаги по получению ипотеки, которые могут отличаться в зависимости о